In [1]:
import psycopg2
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
from geoalchemy2 import Geometry, WKTElement

In [2]:
precinct_shapefile = gpd.read_file("/Users/codyschank/Dropbox/Insight/VTDs/VTDs.shp", encoding = 'utf-8')
#SRID is 3081, checked in QGIS
#project to geographic coordinates to match openaddresses
precinct_shapefile = precinct_shapefile.to_crs({'init': 'epsg:4326'})
precinct_shapefile.columns = map(str.lower, precinct_shapefile.columns)

In [3]:
# Define a database name (we're using a dataset on births, so we'll call it birth_db)
# Set your postgres username
dbname = 'map_the_vote'
username = 'codyschank' # change this to your username

engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)

## create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

postgres://codyschank@localhost/map_the_vote
True


In [4]:
# I don't know why this is necessary, but it is 
precinct_shapefile['geom'] = precinct_shapefile['geometry'].apply(lambda x: WKTElement(x.wkt, srid=4326))
precinct_shapefile.drop('geometry', 1, inplace=True)

In [5]:
table_name = "vtds_tx"
#had to run this from command line first: psql mydatabasename -c "CREATE EXTENSION postgis";
precinct_shapefile.to_sql(table_name, engine, if_exists='replace', index=False, 
                                dtype={'geom': Geometry('POLYGON', srid= 4326)})